In [2]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
data_dir = ('./data/')
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
                "hot_dog", "lightning", "mountain", "skull",
                "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [4]:
X = []
y = []

In [5]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [6]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Max Normalization

In [132]:
# # Normalize pixel values to range 0–1 (from 0–255)
# X = X.astype('float32') / 255.0
# X.shape

In [133]:
# Flatten each image from 28x28 → 784 for dense layers
# X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [134]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [141]:
model = Sequential([
    Input(shape=(784,), name="input_vector"),
    Dense(784, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.4),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.35),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.25),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(len(class_names), activation='linear')
])

In [143]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [144]:
# Train
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [145]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=60, callbacks=callbacks)

Epoch 1/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 194s 13ms/step - accuracy: 0.6756 - loss: 2.1104 - val_accuracy: 0.7906 - val_loss: 1.3634 - learning_rate: 0.0010
Epoch 2/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 194s 14ms/step - accuracy: 0.7537 - loss: 1.4909 - val_accuracy: 0.8058 - val_loss: 1.2571 - learning_rate: 0.0010
Epoch 3/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 194s 14ms/step - accuracy: 0.7603 - loss: 1.4274 - val_accuracy: 0.8186 - val_loss: 1.1939 - learning_rate: 0.0010
Epoch 4/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 192s 13ms/step - accuracy: 0.7647 - loss: 1.3973 - val_accuracy: 0.8202 - val_loss: 1.1709 - learning_rate: 0.0010
Epoch 5/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 193s 14ms/step - accuracy: 0.7655 - loss: 1.3844 - val_accuracy: 0.8227 - val_loss: 1.1663 - learning_rate: 0.0010
Epoch 6/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 194s 14ms/step - accuracy: 0.7671 - loss: 1.3784 - val_accuracy: 0.8200 - val_loss: 1.1557 - learning_rate: 0.0010
Epoch 7/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 19

In [ ]:
y_cv_pred_probs = model.predict(X_cv)   #predict probability for each label
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)  # selects the label with maximum probability

2063/2063 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [ ]:
print(accuracy_score(y_cv,y_cv_pred))

0.865560606060606


In [ ]:
y_test_pred_probs = model.predict(X_test) 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

2063/2063 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
0.8652878787878788


In [ ]:
model.save("doodle_model.keras")

In [9]:
import joblib

data_scaler = {"scaler" : scaler}

joblib.dump(data_scaler, "data_scaler.pkl")

['data_scaler.pkl']